In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import seaborn as sns

# Load data

In [ ]:
data_dir = "../data/raw/"
group1_name = "st01253_by4742swain"

In [ ]:
filepath1 = data_dir + group1_name
timeseries1_filepath = filepath1 + "_timeseries.csv"
labels1_filepath = filepath1 + "_labels.csv"

timeseries_df = pd.read_csv(timeseries1_filepath, index_col=[0,1,2])
labels_df = pd.read_csv(labels1_filepath, index_col=[0,1,2])

In [ ]:
timeseries_df

In [ ]:
timeseries_dropna = timeseries_df.dropna()

In [ ]:
labels_df = labels_df == 1

# Compute periodogram

## Using aliby (inflexible)

In [ ]:
from postprocessor.core.processes.autoreg import autoreg

**TODO: Do on whole dataset** (time-consuming)

In [ ]:
freqs_df, power_df, order_df = autoreg.as_function(timeseries_dropna.iloc[0:5])

## Using decomposed functions

### Fit time series

In [ ]:
from src.autoreg.autoreg import fit_autoreg, optimise_ar_order, autoreg_periodogram

In [ ]:
timeseries = timeseries_dropna.iloc[3].to_numpy()

In [ ]:
order = 17
params = fit_autoreg(timeseries, order)

In [ ]:
params

In [ ]:
model = np.zeros(len(timeseries))
for idx, tp in enumerate(timeseries):
    if idx < order:
        model[idx] = timeseries[idx]
    else:
        preceding_points = timeseries[idx-order:idx]
        linear_combination = np.dot(params["ar_coeffs"][1::], preceding_points[::-1])
        model[idx] = linear_combination

In [ ]:
fig, ax = plt.subplots(figsize=(10,3))
ax.plot(timeseries, label='Original')
ax.plot(model, label=f'Model (order {order})')
ax.legend(loc="lower left")
ax.set_xlabel("Time point")
ax.set_ylabel("Flavin autofluorescence,\n normalised (AU)")

### Optimise order

In [ ]:
max_order = int(3 * np.sqrt(len(timeseries)))
opt_order = optimise_ar_order(timeseries, max_order)

In [ ]:
print(max_order)
print(opt_order)

### Periodogram

In [ ]:
freqs, power = autoreg_periodogram(timeseries, sampling_period=5, freq_npoints=100, ar_order=opt_order)

In [ ]:
plt.plot(freqs, power)

In [ ]:
print(np.max(power))

# Get classifications

Oscillatory means max power > 1

In [ ]:
power_df.max(axis=1)